In [1]:
import os
import numpy as np
from scipy.io import loadmat,savemat
import nibabel as nib
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import scipy.io as sio

bs_size = 1000

# v2: seperate training and testing and save as text
# v3: include all 10 datasets and concatenate!

In [2]:
dirs = ['A','B','C','D','E','F','G','I','J','K']

X = np.empty((3,3,3,15,0))
Y = np.empty((1,1,1,15,0))

for i in range(len(dirs)):
    data_path = "/Users/kurtschilling/Data/harmonization/s%s_A2C_Patches.mat" % (dirs[i])
    print(data_path)
    mat_contents = sio.loadmat(data_path)
    # mat_contents
    Xv = mat_contents['input1200']
    print(X.shape)
    Yv = mat_contents['output1200']
    print(Yv.shape)
    print(Xv.dtype)
    print(Yv.dtype)
    
    # remove NAN, INF
    from numpy import asarray as ar

    arr1 = np.squeeze(np.isnan(Yv).any(axis=3))
    arr2 = np.squeeze(np.isinf(Yv).any(axis=3))
    arr3 = ar(arr1) | ar(arr2)
    print(arr3.shape)

    #X = np.delete(X,arr3,4)
    #Y = np.delete(Y,arr3,4)
    Xv = Xv[:,:,:,:,~arr3]
    Yv = Yv[:,:,:,:,~arr3]
    print(Xv.shape)
    print(Yv.shape)

    # remove >10
    arr6 = np.squeeze(np.greater(np.abs(Yv),10).any(axis=3))
    print(arr6.shape)
    #X = np.delete(X,arr6,4)
    #Y = np.delete(Y,arr6,4)
    Xv = Xv[:,:,:,:,~arr6]
    Yv = Yv[:,:,:,:,~arr6]
    print(Xv.shape)
    print(Yv.shape)

    X = np.append(X, Xv, axis=4)
    Y = np.append(Y, Yv, axis=4)

/Users/kurtschilling/Data/harmonization/sA_A2C_Patches.mat
(3, 3, 3, 15, 0)
(1, 1, 1, 15, 303789)
float64
float64
(303789,)
(3, 3, 3, 15, 303786)
(1, 1, 1, 15, 303786)
(303786,)
(3, 3, 3, 15, 303760)
(1, 1, 1, 15, 303760)
/Users/kurtschilling/Data/harmonization/sB_A2C_Patches.mat
(3, 3, 3, 15, 303760)
(1, 1, 1, 15, 336090)
float64
float64
(336090,)
(3, 3, 3, 15, 336087)
(1, 1, 1, 15, 336087)
(336087,)
(3, 3, 3, 15, 336061)
(1, 1, 1, 15, 336061)
/Users/kurtschilling/Data/harmonization/sC_A2C_Patches.mat
(3, 3, 3, 15, 639821)
(1, 1, 1, 15, 318295)
float64
float64
(318295,)
(3, 3, 3, 15, 318279)
(1, 1, 1, 15, 318279)
(318279,)
(3, 3, 3, 15, 318237)
(1, 1, 1, 15, 318237)
/Users/kurtschilling/Data/harmonization/sD_A2C_Patches.mat
(3, 3, 3, 15, 958058)
(1, 1, 1, 15, 297370)
float64
float64
(297370,)
(3, 3, 3, 15, 297364)
(1, 1, 1, 15, 297364)
(297364,)
(3, 3, 3, 15, 297342)
(1, 1, 1, 15, 297342)
/Users/kurtschilling/Data/harmonization/sE_A2C_Patches.mat
(3, 3, 3, 15, 1255400)
(1, 1, 1, 15, 3

In [3]:
print(X.shape)
print(Y.shape)
print(np.amax(X.shape))
print(np.amax(Xv.shape))

(3, 3, 3, 15, 3208914)
(1, 1, 1, 15, 3208914)
3208914
321717


In [4]:
from torch.utils.data import Dataset
from torchvision import transforms

class SHDataSet(Dataset):
    def __init__(self,X,Y):
        
        self.X = X
        self.Y = Y
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return np.amax(X.shape)
    
    def __getitem__(self, i):
        
        #vec_a = np.reshape(self.X[i,:],(15, 1, 1))
        #vec_b = np.reshape(self.Y[i,:],(15, 1, 1))
        vec_a = self.X[:,:,:,:,i]
        vec_b = self.Y[:,:,:,:,i]
        #vec_a = np.reshape(self.X[i,:],(1, 15, 1))
        #vec_b = np.reshape(self.Y[i,:],(1, 15, 1))
        vec_a = np.transpose(vec_a, (3, 0, 1, 2))
        vec_b = np.transpose(vec_b, (3, 0, 1, 2))
        #a = self.to_tensor(vec_a)
        #b = self.to_tensor(vec_b)
        a = torch.Tensor(vec_a)
        b = torch.Tensor(vec_b.squeeze())
    
        return a,b
    


In [5]:
shset = SHDataSet(X,Y)
print(len(shset))

3208914


In [6]:
print(X.shape)
print(Y.shape)
print(X.dtype)
print(Y.dtype)

(3, 3, 3, 15, 3208914)
(1, 1, 1, 15, 3208914)
float64
float64


In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(3 * 3 * 3 * 15, 600)
        self.cn1 = nn.Conv3d(15,128,kernel_size=(3,3,3),stride=1,padding=(1,1,1))
        #print(self.cn1.shape)
        #self.fc2 = nn.Linear(600,300)
        self.bn = nn.BatchNorm3d(128)
        self.fc2 = nn.Linear(128*3*3*3, 300)
        self.fc3 = nn.Linear(300,60)
        self.fc4 = nn.Linear(60,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        #print(x.shape)
        x = F.relu(self.cn1(x))
        #print(x.shape)
        dimensions = x.shape
        x = self.bn(x)
        x = x.view(dimensions[0], -1)
        #x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net)  

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.MSELoss()

Net(
  (cn1): Conv3d(15, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=3456, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=60, bias=True)
  (fc4): Linear(in_features=60, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [8]:
from __future__ import print_function, division
import torch

def train(model, device, loader, optimizer):
    model.train()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()
        
        optimizer.zero_grad()
        output = model(data)
        output = output.to(device)

        loss = criterion(output,target)
        total_loss += loss.item()
      
        loss.backward()
        optimizer.step()
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data),len(shset.X),100.*batch_idx/len(shset.X),loss.data[0]))

    avg_loss = total_loss / batch_idx
    print('\tTraining set: Average loss: {:.4f}'.format(avg_loss))
   
    return avg_loss

In [9]:
def test(model, device, loader):
    model.eval()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()

        output = model(data)
        output = output.to(device)

        loss = criterion(output, target)
        total_loss += loss.item()

    avg_loss = total_loss / batch_idx
    print('\tTesting set: Average loss: {:.4f}'.format(avg_loss))

    return avg_loss



In [10]:
# batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42


# Creating data indices for training and validation splits:
dataset_size = len(shset)
print(dataset_size)

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

from torch.utils.data.sampler import SubsetRandomSampler

use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(shset, batch_size=bs_size, sampler=train_sampler)
validation_loader = DataLoader(shset, batch_size=bs_size, sampler=valid_sampler)



3208914


In [11]:
import random

train_loss_file = 'BLAHFCN_PATCH_A2C_SH1200_train_loss_split.txt'
f = open(train_loss_file, 'w')
f.close()
validate_loss_file = 'BLAHFCN_PATCH_A2C_SH1200_validate_loss_split.txt'
f = open(validate_loss_file, 'w')
f.close()

model_file = 'BLAHFCN_PATCH_A2C_SH1200_saved_model_split'

In [ ]:
for epoch in range(1, 51):
        print('\nEpoch %d: ' % epoch)
        loss = train(net, device, train_loader, optimizer)

        with open(train_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        loss = test(net, device, validation_loader)

        with open(validate_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        if epoch % 5 == 0:
            with open(model_file, 'wb') as f:
                torch.save(net.state_dict(), f)


Epoch 1: 


/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/3 (0%)]	Loss: 0.122147
Train Epoch: 1 [1000/3 (33%)]	Loss: 0.118952
Train Epoch: 1 [2000/3 (67%)]	Loss: 0.117829
Train Epoch: 1 [3000/3 (100%)]	Loss: 0.117087
Train Epoch: 1 [4000/3 (133%)]	Loss: 0.116871
Train Epoch: 1 [5000/3 (167%)]	Loss: 0.115979
Train Epoch: 1 [6000/3 (200%)]	Loss: 0.118444
Train Epoch: 1 [7000/3 (233%)]	Loss: 0.117226
Train Epoch: 1 [8000/3 (267%)]	Loss: 0.119560
Train Epoch: 1 [9000/3 (300%)]	Loss: 0.116503
Train Epoch: 1 [10000/3 (333%)]	Loss: 0.114839
Train Epoch: 1 [11000/3 (367%)]	Loss: 0.110572
Train Epoch: 1 [12000/3 (400%)]	Loss: 0.113145
Train Epoch: 1 [13000/3 (433%)]	Loss: 0.107786
Train Epoch: 1 [14000/3 (467%)]	Loss: 0.113813
Train Epoch: 1 [15000/3 (500%)]	Loss: 0.109494
Train Epoch: 1 [16000/3 (533%)]	Loss: 0.107410
Train Epoch: 1 [17000/3 (567%)]	Loss: 0.102502
Train Epoch: 1 [18000/3 (600%)]	Loss: 0.105453
Train Epoch: 1 [19000/3 (633%)]	Loss: 0.102782
Train Epoch: 1 [20000/3 (667%)]	Loss: 0.102799
Train Epoch: 1 [21000/3 (700%)

Train Epoch: 1 [171000/3 (5700%)]	Loss: 0.008009
Train Epoch: 1 [172000/3 (5733%)]	Loss: 0.007880
Train Epoch: 1 [173000/3 (5767%)]	Loss: 0.008311
Train Epoch: 1 [174000/3 (5800%)]	Loss: 0.007612
Train Epoch: 1 [175000/3 (5833%)]	Loss: 0.008124
Train Epoch: 1 [176000/3 (5867%)]	Loss: 0.010861
Train Epoch: 1 [177000/3 (5900%)]	Loss: 0.007937
Train Epoch: 1 [178000/3 (5933%)]	Loss: 0.007533
Train Epoch: 1 [179000/3 (5967%)]	Loss: 0.008037
Train Epoch: 1 [180000/3 (6000%)]	Loss: 0.008154
Train Epoch: 1 [181000/3 (6033%)]	Loss: 0.008096
Train Epoch: 1 [182000/3 (6067%)]	Loss: 0.007912
Train Epoch: 1 [183000/3 (6100%)]	Loss: 0.011006
Train Epoch: 1 [184000/3 (6133%)]	Loss: 0.007827
Train Epoch: 1 [185000/3 (6167%)]	Loss: 0.008069
Train Epoch: 1 [186000/3 (6200%)]	Loss: 0.008228
Train Epoch: 1 [187000/3 (6233%)]	Loss: 0.009818
Train Epoch: 1 [188000/3 (6267%)]	Loss: 0.008227
Train Epoch: 1 [189000/3 (6300%)]	Loss: 0.008082
Train Epoch: 1 [190000/3 (6333%)]	Loss: 0.007820
Train Epoch: 1 [1910

Train Epoch: 1 [338000/3 (11267%)]	Loss: 0.008015
Train Epoch: 1 [339000/3 (11300%)]	Loss: 0.007518
Train Epoch: 1 [340000/3 (11333%)]	Loss: 0.007495
Train Epoch: 1 [341000/3 (11367%)]	Loss: 0.012786
Train Epoch: 1 [342000/3 (11400%)]	Loss: 0.007020
Train Epoch: 1 [343000/3 (11433%)]	Loss: 0.010060
Train Epoch: 1 [344000/3 (11467%)]	Loss: 0.011870
Train Epoch: 1 [345000/3 (11500%)]	Loss: 0.007392
Train Epoch: 1 [346000/3 (11533%)]	Loss: 0.007612
Train Epoch: 1 [347000/3 (11567%)]	Loss: 0.007378
Train Epoch: 1 [348000/3 (11600%)]	Loss: 0.007733
Train Epoch: 1 [349000/3 (11633%)]	Loss: 0.009276
Train Epoch: 1 [350000/3 (11667%)]	Loss: 0.010011
Train Epoch: 1 [351000/3 (11700%)]	Loss: 0.007719
Train Epoch: 1 [352000/3 (11733%)]	Loss: 0.008252
Train Epoch: 1 [353000/3 (11767%)]	Loss: 0.007525
Train Epoch: 1 [354000/3 (11800%)]	Loss: 0.007525
Train Epoch: 1 [355000/3 (11833%)]	Loss: 0.007434
Train Epoch: 1 [356000/3 (11867%)]	Loss: 0.007627
Train Epoch: 1 [357000/3 (11900%)]	Loss: 0.007195


...

Train Epoch: 15 [1054000/3 (35133%)]	Loss: 0.002688
Train Epoch: 15 [1055000/3 (35167%)]	Loss: 0.002687
Train Epoch: 15 [1056000/3 (35200%)]	Loss: 0.003155
Train Epoch: 15 [1057000/3 (35233%)]	Loss: 0.002850
Train Epoch: 15 [1058000/3 (35267%)]	Loss: 0.003153
Train Epoch: 15 [1059000/3 (35300%)]	Loss: 0.002694
Train Epoch: 15 [1060000/3 (35333%)]	Loss: 0.002505
Train Epoch: 15 [1061000/3 (35367%)]	Loss: 0.003712
Train Epoch: 15 [1062000/3 (35400%)]	Loss: 0.002822
Train Epoch: 15 [1063000/3 (35433%)]	Loss: 0.003075
Train Epoch: 15 [1064000/3 (35467%)]	Loss: 0.006122
Train Epoch: 15 [1065000/3 (35500%)]	Loss: 0.003247
Train Epoch: 15 [1066000/3 (35533%)]	Loss: 0.002488
Train Epoch: 15 [1067000/3 (35567%)]	Loss: 0.003081
Train Epoch: 15 [1068000/3 (35600%)]	Loss: 0.003866
Train Epoch: 15 [1069000/3 (35633%)]	Loss: 0.002532
Train Epoch: 15 [1070000/3 (35667%)]	Loss: 0.002945
Train Epoch: 15 [1071000/3 (35700%)]	Loss: 0.003332
Train Epoch: 15 [1072000/3 (35733%)]	Loss: 0.002706
Train Epoch:

Train Epoch: 15 [1212000/3 (40400%)]	Loss: 0.002913
Train Epoch: 15 [1213000/3 (40433%)]	Loss: 0.002810
Train Epoch: 15 [1214000/3 (40467%)]	Loss: 0.003102
Train Epoch: 15 [1215000/3 (40500%)]	Loss: 0.003085
Train Epoch: 15 [1216000/3 (40533%)]	Loss: 0.005273
Train Epoch: 15 [1217000/3 (40567%)]	Loss: 0.003102
Train Epoch: 15 [1218000/3 (40600%)]	Loss: 0.003866
Train Epoch: 15 [1219000/3 (40633%)]	Loss: 0.002871
Train Epoch: 15 [1220000/3 (40667%)]	Loss: 0.003569
Train Epoch: 15 [1221000/3 (40700%)]	Loss: 0.002809
Train Epoch: 15 [1222000/3 (40733%)]	Loss: 0.005147
Train Epoch: 15 [1223000/3 (40767%)]	Loss: 0.004714
Train Epoch: 15 [1224000/3 (40800%)]	Loss: 0.002857
Train Epoch: 15 [1225000/3 (40833%)]	Loss: 0.002807
Train Epoch: 15 [1226000/3 (40867%)]	Loss: 0.003068
Train Epoch: 15 [1227000/3 (40900%)]	Loss: 0.002790
Train Epoch: 15 [1228000/3 (40933%)]	Loss: 0.002959
Train Epoch: 15 [1229000/3 (40967%)]	Loss: 0.002615
Train Epoch: 15 [1230000/3 (41000%)]	Loss: 0.002774
Train Epoch:

Train Epoch: 15 [1370000/3 (45667%)]	Loss: 0.003276
Train Epoch: 15 [1371000/3 (45700%)]	Loss: 0.003624
Train Epoch: 15 [1372000/3 (45733%)]	Loss: 0.003578
Train Epoch: 15 [1373000/3 (45767%)]	Loss: 0.002897
Train Epoch: 15 [1374000/3 (45800%)]	Loss: 0.002647
Train Epoch: 15 [1375000/3 (45833%)]	Loss: 0.002799
Train Epoch: 15 [1376000/3 (45867%)]	Loss: 0.002626
Train Epoch: 15 [1377000/3 (45900%)]	Loss: 0.003061
Train Epoch: 15 [1378000/3 (45933%)]	Loss: 0.005588
Train Epoch: 15 [1379000/3 (45967%)]	Loss: 0.003127
Train Epoch: 15 [1380000/3 (46000%)]	Loss: 0.003307
Train Epoch: 15 [1381000/3 (46033%)]	Loss: 0.002605
Train Epoch: 15 [1382000/3 (46067%)]	Loss: 0.002988
Train Epoch: 15 [1383000/3 (46100%)]	Loss: 0.002567
Train Epoch: 15 [1384000/3 (46133%)]	Loss: 0.003186
Train Epoch: 15 [1385000/3 (46167%)]	Loss: 0.002568
Train Epoch: 15 [1386000/3 (46200%)]	Loss: 0.003093
Train Epoch: 15 [1387000/3 (46233%)]	Loss: 0.003342
Train Epoch: 15 [1388000/3 (46267%)]	Loss: 0.003071
Train Epoch:

Train Epoch: 15 [1528000/3 (50933%)]	Loss: 0.004323
Train Epoch: 15 [1529000/3 (50967%)]	Loss: 0.002716
Train Epoch: 15 [1530000/3 (51000%)]	Loss: 0.003099
Train Epoch: 15 [1531000/3 (51033%)]	Loss: 0.002545
Train Epoch: 15 [1532000/3 (51067%)]	Loss: 0.003147
Train Epoch: 15 [1533000/3 (51100%)]	Loss: 0.002803
Train Epoch: 15 [1534000/3 (51133%)]	Loss: 0.002798
Train Epoch: 15 [1535000/3 (51167%)]	Loss: 0.005628
Train Epoch: 15 [1536000/3 (51200%)]	Loss: 0.004812
Train Epoch: 15 [1537000/3 (51233%)]	Loss: 0.003366
Train Epoch: 15 [1538000/3 (51267%)]	Loss: 0.002753
Train Epoch: 15 [1539000/3 (51300%)]	Loss: 0.003359
Train Epoch: 15 [1540000/3 (51333%)]	Loss: 0.002467
Train Epoch: 15 [1541000/3 (51367%)]	Loss: 0.002683
Train Epoch: 15 [1542000/3 (51400%)]	Loss: 0.005431
Train Epoch: 15 [1543000/3 (51433%)]	Loss: 0.002702
Train Epoch: 15 [1544000/3 (51467%)]	Loss: 0.002857
Train Epoch: 15 [1545000/3 (51500%)]	Loss: 0.006940
Train Epoch: 15 [1546000/3 (51533%)]	Loss: 0.002910
Train Epoch:

Train Epoch: 15 [1686000/3 (56200%)]	Loss: 0.002639
Train Epoch: 15 [1687000/3 (56233%)]	Loss: 0.002908
Train Epoch: 15 [1688000/3 (56267%)]	Loss: 0.003078
Train Epoch: 15 [1689000/3 (56300%)]	Loss: 0.003971
Train Epoch: 15 [1690000/3 (56333%)]	Loss: 0.003197
Train Epoch: 15 [1691000/3 (56367%)]	Loss: 0.002422
Train Epoch: 15 [1692000/3 (56400%)]	Loss: 0.002635
Train Epoch: 15 [1693000/3 (56433%)]	Loss: 0.003157
Train Epoch: 15 [1694000/3 (56467%)]	Loss: 0.003019
Train Epoch: 15 [1695000/3 (56500%)]	Loss: 0.003405
Train Epoch: 15 [1696000/3 (56533%)]	Loss: 0.002655
Train Epoch: 15 [1697000/3 (56567%)]	Loss: 0.002805
Train Epoch: 15 [1698000/3 (56600%)]	Loss: 0.002861
Train Epoch: 15 [1699000/3 (56633%)]	Loss: 0.002661
Train Epoch: 15 [1700000/3 (56667%)]	Loss: 0.002901
Train Epoch: 15 [1701000/3 (56700%)]	Loss: 0.007339
Train Epoch: 15 [1702000/3 (56733%)]	Loss: 0.003322
Train Epoch: 15 [1703000/3 (56767%)]	Loss: 0.002841
Train Epoch: 15 [1704000/3 (56800%)]	Loss: 0.004206
Train Epoch: